In [1]:
import os
from os import listdir
from PIL import Image
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
from keras_facenet import FaceNet
import pickle
import cv2

In [2]:
#HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = FaceNet()

In [3]:
folder='dataset/'
database = {}

for filename in listdir(folder):

    path = folder + filename
    gbr1 = cv2.imread(folder + filename)
    
    wajah = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
    if len(wajah)>0:
        x1, y1, width, height = wajah[0]         
    else:
        x1, y1, width, height = 1, 1, 10, 10
        
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  # konversi dari OpenCV ke PIL
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
     
    face = expand_dims(face, axis=0)
    signature = MyFaceNet.embeddings(face)
    
    database[os.path.splitext(filename)[0]]=signature


1/1 [==============================] - 0s 77ms/step


In [4]:
myfile = open("data_ori.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

In [5]:
database

{'akbar': array([[ 0.02747555,  0.05438858, -0.04526115,  0.06469756, -0.01635289,
         -0.03941442,  0.01821971,  0.01749491,  0.07959941,  0.00731274,
          0.07340635,  0.03796624,  0.05258681, -0.0011027 , -0.07661933,
          0.0698701 ,  0.09521798,  0.04471146, -0.0320231 ,  0.03646445,
         -0.02163315, -0.06784105,  0.0263862 , -0.0552809 , -0.01707185,
         -0.03699557, -0.00918218, -0.01650735,  0.0204759 ,  0.08123762,
          0.01316519,  0.02792055, -0.03877309, -0.00615101, -0.02138141,
         -0.03269545, -0.02090232, -0.09372273, -0.06969874,  0.07652745,
         -0.05303426, -0.03442654, -0.03628751,  0.04033491, -0.04245575,
          0.01956186, -0.0025062 ,  0.08726545,  0.0240689 ,  0.0113418 ,
         -0.04313963, -0.10134675,  0.02162357,  0.02217587,  0.0143698 ,
         -0.02069106, -0.04047128,  0.00657112,  0.02409201,  0.02147887,
          0.02263692,  0.0447786 , -0.00566223,  0.07085352,  0.00519106,
          0.01495098, -0.0229

In [6]:
cap = cv2.VideoCapture(0)

while(1):
    _, gbr1 = cap.read()
    
    wajah = HaarCascade.detectMultiScale(gbr1, 1.1, 4)
     
    if len(wajah) > 0:
        x1, y1, width, height = wajah[0]        
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        
        gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
        gbr = Image.fromarray(gbr)  # konversi dari OpenCV ke PIL
        gbr_array = asarray(gbr)
        
        face = gbr_array[y1:y2, x1:x2]                        
        face = Image.fromarray(face)                       
        face = face.resize((160, 160))
        face = asarray(face)
        
        face = expand_dims(face, axis=0)
        signature = MyFaceNet.embeddings(face)
            
        min_dist = 100
        identity = ' '
        for key, value in database.items():
            dist = np.linalg.norm(value-signature)
            if dist < min_dist:
                min_dist = dist
                identity = key
            
        cv2.putText(gbr1, f'Hallo {identity} selamat datang', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)
    else:
        cv2.putText(gbr1, 'Wajah tidak terdeteksi', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0 ,0, 0), 2, cv2.LINE_AA)
    
    cv2.imshow('res', gbr1)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 78ms/step
